State - [1,2,3,4,5,6,7,8] for n=8, i.e. 8x8 board, where 1 represents the position of the first column's queen, 2 represents the position of the second column's queen, etc.

In [3]:
import random
import numpy as np

In [4]:
def fitness(numberOfCollisions):
    """
    Function fitness
    Takes as input numberOfCollisions
    Outputs a number between (0:1], where 1 means 0 collisions
      
    """

    return 1/(1+numberOfCollisions)

def checkDiagonalCollisions(matrix):
    """
    Function checkDiagonalCollisions
    Takes as input matrix representation of the state
    Outputs the number of diagonal collisions  
    
    """

    diags = [matrix[::-1,:].diagonal(i) for i in range(-(matrix.shape[0]-1),matrix.shape[0])] # Get the diagonals of one direction

    diags.extend(matrix.diagonal(i) for i in range(matrix.shape[0]-1,-matrix.shape[0],-1)) # Get the diagonals of second direction

    arr = np.array([(n.sum()*(n.sum()-1))/2 for n in diags]) #Find number of collisions for every diagonal in a matrix

    arr = np.where(arr<0,0,arr) #Fill negative values with 0

    return arr.sum() #Sum values to get 


def findNofCollisions(state):
    """
    Function findNofCollisions
    Takes as input the state
    Outputs the number of collisions of that state
    
    """

    state = np.array(state) - 1 #Get indices
    n = len(state) #Length of the state
    matrix = np.zeros((n,n)) #Create the matrix
    matrix[np.arange(n), state] = 1 #Insert ones for the indices of the queens

    num_of_DiagCollisions = checkDiagonalCollisions(matrix) #Check for the collisions on diagonal

    return num_of_DiagCollisions

def initPopulation(n, p_size):
    """
    Function initPopulation
    Takes as input the number of queens and population size
    Outputs the population with n-length states 
    
    """

    population = np.array([np.arange(n)+1 for i in range(p_size)]) #Initialize p_size times n-length aranged state

    np.apply_along_axis(np.random.shuffle, 1, population) #Randomly shuffle the states to get random states

    return population

def selectParents(population, fitnesses):    #Stochastic Universal Sampling 
    """
    Function selectParents
    Method - Stochastic Universal Sampling
    Takes as input population and fitnesses
    Outputs choosen pair of parents
    
    """

    indices = np.arange(len(population))   #Get indices of populations

    probability = fitnesses/sum(fitnesses)   #Find the probability of fitnesses to be selected

    choice = np.random.choice(indices, 2, p = probability, replace=False)   #Randomly choose two parents 

    return population[choice]

def crossover(parents_pair):
    """
    Function crossover
    Method - Order One Crossover
    Takes as input parents pairs
    Outputs produced offsprings
    
    """

    parent1 = parents_pair[0]    #Parent one

    parent2 = parents_pair[1]    #Parent two

    length = len(parent1)    #Length of the parents

    point1 = np.random.randint(length)    #Generate first point

    point2 = np.random.randint(length)    #Generate second point

    minimum = min(point1, point2)    #Take the minimum of the points

    maximum = max(point1, point2)    #Take the maximum of the points

    #Separate the parents into segments
    p_1_segment_1 = parent1[:minimum]
    p_1_segment_2 = parent1[minimum:maximum]
    p_1_segment_3 = parent1[maximum:]
    p_2_segment_1 = parent2[:minimum]
    p_2_segment_2 = parent2[minimum:maximum]
    p_2_segment_3 = parent2[maximum:]
    

    point2parent1 = np.concatenate((p_1_segment_3, p_1_segment_1, p_1_segment_2), axis=None)    #Shuffle the segments in order 3,1,2
    
    point2parent2 = np.concatenate((p_2_segment_3, p_2_segment_1, p_2_segment_2), axis=None)    #Shuffle the segments in order 3,1,2

    p_1_segment_2, p_2_segment_2 = p_2_segment_2, p_1_segment_2    #Change the segment 2 of parent1 with parent2 and vice versa

    #Subtract new middle segment's elements from parent to get remained elements 
    p_1_remainder = point2parent1[np.invert(np.isin(point2parent1, p_1_segment_2))]   
    p_2_remainder = point2parent2[np.invert(np.isin(point2parent2, p_2_segment_2))]

    #Create new segments from remainders
    p_1_segment_3 = p_1_remainder[:len(p_1_segment_3)]
    p_1_segment_1 = p_1_remainder[len(p_1_segment_3):]

    p_2_segment_3 = p_2_remainder[:len(p_2_segment_3)]
    p_2_segment_1 = p_2_remainder[len(p_2_segment_3):]
    
    #Get new offsprings by concatenating processed segments
    offspring1 = np.concatenate((p_1_segment_1, p_1_segment_2, p_1_segment_3), axis=None)
    offspring2 = np.concatenate((p_2_segment_1, p_2_segment_2, p_2_segment_3), axis=None)

    return offspring1, offspring2


def findSolution(board_size = 8, max_generation = 50, num_of_offsprings = 10, mut_prob = 0.5):
    """
    Function findSolution
    Takes as input board_size, max_generaion, number of offsprings and mutation probability
    Outputs the best solution
    
    """

    done = False    #if done

    generation = 0    #Generation count

    population = initPopulation(board_size,8)   #Initialize population

    nofcollisions = list(map(findNofCollisions, population))   #Compute the n of collisions for the population

    fitnesses = fitness(np.array(nofcollisions))    #Calculate the fitnesses for the collisions for the population

    print('Initialized Population with fitnesses')
    print(population)
    print(fitnesses)

    while generation <= max_generation:    #While the max generation count is not reached

        print('-'*50)

        print('Generation number: ' + str(generation))

        print('Population and fitnesses for generation:')

        print(population)
        print(fitnesses)


        for i in range(int(num_of_offsprings/2)):    #Generate num_of_offsprings/2 offsprings

            pair = selectParents(population, fitnesses)    #Select two pairs from population

            offspring1, offspring2 = crossover(pair)    #Produce offsprings

            print(offspring2)

            offspring1 = mutation(offspring1, mut_prob)   #Mutate the first offspring

            offspring2 = mutation(offspring2, mut_prob)    #Mutate the second offspring
 
            print(offspring2)

            fitness_1 = fitness(findNofCollisions(offspring1))    #Calculate the fitness for the first offspring
            fitness_2 = fitness(findNofCollisions(offspring2))    #Calculate the fitness for the second offspring

            print('New pair of offsprings generated:')
            print(offspring1)
            print(offspring2)
            print('Fitnesses')
            print(fitness_1)
            print(fitness_2)

            indices_of_highest_fitness = np.argsort(fitnesses)   #Find the indices of the highest fitnesses

            sorted_fitness = fitnesses[indices_of_highest_fitness]    #Get the sorted fitness

            sorted_population = population[indices_of_highest_fitness]   #Get the sorted population

            print('Sorted Population')
            print(population)
            print('Sorted Fitnesses')
            print(sorted_fitness)

            is_greater = np.greater(np.array([fitness_1, fitness_2]), sorted_fitness[:2])    #If the offsprings are greater than the lowest of the population

            if is_greater.all():   #If both are greater

                print('Both offsprings are better than the worst individuals')

                sorted_fitness[:2] = np.array([fitness_1, fitness_2])    #Replace the two lowest with offsprings
                sorted_population[:2] = np.array([offspring1, offspring2])

            elif is_greater.any():   #If one of them is greater

                print('One offspring is better than the worst individual')

                index = np.argmax(np.array([fitness_1, fitness_2]))   #Get the index of the highest fitness

                sorted_fitness[:1] = np.array([fitness_1, fitness_2])[index]   #Replace the lowest individual with new offspring
                sorted_population[:1] = np.array([offspring1, offspring2])[index]

            else:

                print('None of the offsprings is better')

                continue

            population = sorted_population   

            fitnesses = sorted_fitness

            print('New population')
            print(population)
            print('New fitnesses')
            print(fitnesses)

            maximum_for_generation = np.argmax(fitnesses)   #Get the maximum fitness for the generation

            print('Maximum fitness for generation')
            print(fitnesses[maximum_for_generation])

            if fitnesses[maximum_for_generation] == 1.0:    #If the maximum is 1, then terminate the algorithm

                print('-'*50)
                print('****TERMINATING ALGORITHM****')
                print('-'*50)

                print('The goal state reached')
                state = population[maximum_for_generation]
                print(state)
                print('Generation - ' + str(generation))
                
                #Get the matrix form of the state
                state = np.array(state) - 1
                n = len(state)
                matrix = np.zeros((n,n))
                matrix[np.arange(n), state] = 1

                print('Solution board')
                print(' ')
                print(matrix)
                print(' ')
                print('-'*50)

                done = True

                break
        if done:

            break


            print('-'*50)

        generation += 1

    print('Maximum for generation end')
    print(fitnesses[maximum_for_generation])
    
    return population[maximum_for_generation]


def mutation(state, prob):
    """
    Function mutation
    Method - Swap Mutation
    Takes as input state and probability
    Outputs mutated state if mutations is done
    
    """

    number = random.randrange(0, 100)    #Get a number in range of 100 numbers

    if number < prob*100:   #If the number is lower than the probability then do the mutation

        ix1 = np.random.randint(len(state))    #Get random index 1

        ix2 = np.random.randint(len(state))   #Get random index 2

        state[[ix1, ix2]] = state[[ix2, ix1]]   #Swap the values

    return state













In [5]:
findSolution(8, 20, 10, 0.8)

Initialized Population with fitnesses
[[2 5 7 1 8 3 4 6]
 [6 3 5 1 7 4 2 8]
 [2 3 4 5 8 6 1 7]
 [4 8 6 5 7 2 3 1]
 [8 5 1 6 4 2 7 3]
 [2 8 3 7 1 6 4 5]
 [5 1 6 4 3 8 2 7]
 [1 4 3 5 6 7 8 2]]
[0.2        0.33333333 0.14285714 0.14285714 0.5        0.2
 0.25       0.11111111]
--------------------------------------------------
Generation number: 0
Population and fitnesses for generation:
[[2 5 7 1 8 3 4 6]
 [6 3 5 1 7 4 2 8]
 [2 3 4 5 8 6 1 7]
 [4 8 6 5 7 2 3 1]
 [8 5 1 6 4 2 7 3]
 [2 8 3 7 1 6 4 5]
 [5 1 6 4 3 8 2 7]
 [1 4 3 5 6 7 8 2]]
[0.2        0.33333333 0.14285714 0.14285714 0.5        0.2
 0.25       0.11111111]
[3 8 6 5 7 2 4 1]
[3 8 6 5 7 2 4 1]
New pair of offsprings generated:
[2 5 7 1 6 3 4 8]
[3 8 6 5 7 2 4 1]
Fitnesses
0.16666666666666666
0.2
Sorted Population
[[2 5 7 1 8 3 4 6]
 [6 3 5 1 7 4 2 8]
 [2 3 4 5 8 6 1 7]
 [4 8 6 5 7 2 3 1]
 [8 5 1 6 4 2 7 3]
 [2 8 3 7 1 6 4 5]
 [5 1 6 4 3 8 2 7]
 [1 4 3 5 6 7 8 2]]
Sorted Fitnesses
[0.11111111 0.14285714 0.14285714 0.2        0.

array([6, 3, 1, 8, 4, 2, 7, 5])